In [11]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv('data/raw/quakenu_raw.csv')

# print(raw_df[:1])
# print(raw_df.dtypes)

In [30]:
import datetime

cleaning_test_df = raw_df[::1000] # every 1000th post

def clean_post_df(raw_df):
    clean_df = raw_df
    clean_df['date'] = pd.to_datetime(clean_df['date'], format='%Y-%m-%d, %H:%M')
    return clean_df

clean_test_df = clean_post_df(cleaning_test_df)

print(clean_test_df['post_text'][0])

how do i get 500hz in winxp sp1??


C:\Users\BYRONA~1\AppData\Local\Temp/ipykernel_15728/2991618677.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['date'] = pd.to_datetime(clean_df['date'], format='%Y-%m-%d, %H:%M')


In [47]:
import re

"""
Cleaning a single string for NLP
    remove nonchars except for apostrophes

"""

raw_text_col = raw_df['post_text']

print(raw_df.iloc[32])

for test_text in raw_text_col[32:33]:
    print(test_text)
    print(re.sub(r'[^a-z\']',' ',test_text))

author                                                       3
date                                         2006-01-14, 14:01
forum                                                       10
post_id                                                     52
post_text    i will fix it up in parts so u understand ever...
thread                                                      28
Name: 32, dtype: object
i will fix it up in parts so u understand everything!! when u start the config it automatically plays a mp3 song and put   on the  . dont forget to put ur correct winamp dir f8 = tells u the song u play in winamp with   and   u toggle shuffle/repeat with   and   u rewind/forward the song with   and   u play previuse/next song ///// sassa mp3_volume "0.26" mp3_play volume "0" mp3_winamp_dir "c&#58;programwinamp" bind  f1            "at;echo autotrack" bind  f2            "ls;echo lastscores" bind  f3            "sc;echo +scores" bind  f4            "exec mvd.txt" bind  f5            "spectato